# Kaggle Credit Card Fraud Detection Using Neural Networks

## Dataset Description and Notebook Objectives

### Description

From the [Kaggle Dataset](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)'s description: "The dataset contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions. It contains only numerical input variables which are the result of a PCA transformation... due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise."

The dataset is provided in csv format as creditcard.csv. The file can be found on Kaggle at the link provided above, and is not included in this GitHub repository.

### Objectives

Objective: Create a machine learning algorithm that can accurately detect fraudulent transactions.

Given that our dataset contains PCA components, rather than the features themselves, there is little reason (with respect to the end result) to track feature importance. Coupled with the fact that the analyses contained within this repository are for practicing different data science skills, this is an excellent opportunity to leverage use of unsupervised algorithms. More specifically, we will use a neural network.

The Kaggle description recommends measuring accuracy with Area Under the Precision-Recall Curve (AUPRC) because "confusion matrix accuracy is not meaningful for unbalanced classification." Perhaps this is true when using weighted recall (the default CM accuracy metric) but a more preferable metric with respect to the confusion matrix, is to use an unweighted F1-score. For learning purposes, we will explore both Area-Under metrics, as well as this unweighted F1-score.